# Spend Forecast (Prophet)

# **Import packages and querying data**
In this section we are importing the different packages and creating the dataframe from bigquery

In [ ]:

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import holidays
from prophet import Prophet
import pandas_gbq
from datetime import datetime, timedelta

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [ ]:
# Import required libraries
from google.cloud import bigquery

In [ ]:
sql = """
WITH company AS (SELECT * FROM `analytics_companies.companies_current`)
, spend AS (SELECT * FROM `analytics_ssot.spend_per_customer_daily`)

SELECT DATE(s.performed_date)                       AS performed_date
    , CASE WHEN c.company_market = 'GB' THEN c.company_market
           WHEN c.company_market = 'DK' THEN c.company_market
           WHEN c.company_market = 'DE' THEN c.company_market
           WHEN c.company_market = 'ES' THEN c.company_market
           WHEN c.company_market = 'SE' THEN c.company_market
           WHEN c.company_market = 'NL' THEN c.company_market
           WHEN c.company_market = 'FR' THEN c.company_market
           ELSE 'Other Markets' END                         AS country
    , COUNT(DISTINCT(IF(spend_eur > 0, s.company_id,NULL)))                                        AS spending_customer_count
    , SUM(s.spend_eur) / NULLIF( COUNT(DISTINCT(IF(spend_eur > 0, s.company_id,NULL))),0)          AS spend_per_spending_customer
    , SUM(s.spend_eur)                                                                             AS spend
FROM spend AS s
LEFT JOIN company AS c
    ON c.company_id = s.company_id
WHERE 1 = 1
    AND s.performed_date >= '2021-02-01'
GROUP BY 1, 2


"""

In [1]:
client = bigquery.Client("production")

query_job = client.query(sql)

df = query_job.to_dataframe()

NameError: name 'bigquery' is not defined

In [ ]:
df.head()

,performed_date,country,spending_customer_count,spend_per_spending_customer,spend
0,2024-04-18,ES,1366,584.582482,7.985397e+05
1,2022-05-19,GB,4211,614.561064,2.587917e+06
2,2023-01-09,Other Markets,281,442.804126,1.244280e+05
3,2021-12-31,DK,1506,421.420753,6.346597e+05
4,2023-03-25,GB,5219,473.115503,2.469190e+06


# **Cleaning data**
In this section cleaning the data and exploring it so we can input it in the algorithm in the best way

In [ ]:
#Filling null values
df1 = df

df1['spending_customer_count'] = df1['spending_customer_count'].fillna(0)
df1['spend_per_spending_customer'] = df1['spend_per_spending_customer'].fillna(0)
df1['spend'] = df1['spend'].fillna(0)

In [ ]:
# Formatting
df1["performed_date"] = pd.to_datetime(df1["performed_date"])
df1["spending_customer_count"] = pd.to_numeric(df1["spending_customer_count"])
df1["spend_per_spending_customer"] = pd.to_numeric(df1["spend_per_spending_customer"])

Spending Customers show a more linear growth than the Avg Spend per Spending Customer

# **Holidays package**

In [ ]:
# set holidays per country
# dict_keys(['AT', 'BE', 'DE', 'DK', 'ES', 'FI', 'FR', 'GB', 'IE', 'NL', 'Other', 'PT', 'SE'])

holidays_AT = pd.DataFrame({'holiday': 'holidays_AT',
  'ds': pd.to_datetime(['2022-01-01','2022-01-06','2022-04-18','2022-05-01','2022-05-26','2022-06-06','2022-06-16','2022-08-15','2022-10-26','2022-11-01','2022-12-08','2022-12-25','2022-12-26',
                        '2023-01-01','2023-01-06','2023-04-10','2023-05-01','2023-05-18','2023-05-29','2023-06-08','2023-08-15','2023-10-26','2023-11-01','2023-12-08','2023-12-25','2023-12-26',
                        '2024-01-01','2024-01-06','2024-04-01','2024-05-01','2024-05-09','2024-05-20','2024-05-30','2024-08-15','2024-10-26','2024-11-01','2024-12-08','2024-12-25','2024-12-26',
                        '2025-01-01','2025-01-06','2025-04-21','2025-05-01','2025-05-29','2025-06-09','2025-06-19','2025-08-15','2025-10-26','2025-11-01','2025-12-08','2025-12-25','2025-12-26']),
  'lower_window': 0,'upper_window': 0,})

holidays_BE = pd.DataFrame({'holiday': 'holidays_BE',
  'ds': pd.to_datetime(['2022-01-01','2022-04-17','2022-04-18','2022-05-01','2022-05-26','2022-06-06','2022-07-21','2022-08-15','2022-11-01','2022-11-11','2022-12-25',
                        '2023-01-01','2023-04-09','2023-04-10','2023-05-01','2023-05-18','2023-05-29','2023-07-21','2023-08-15','2023-11-01','2023-11-11','2023-12-25',
                        '2024-01-01','2024-03-31','2024-04-01','2024-05-01','2024-05-09','2024-05-20','2024-07-21','2024-08-15','2024-11-01','2024-11-11','2024-12-25',
                        '2025-01-01','2025-04-20','2025-04-21','2025-05-01','2025-05-29','2025-06-09','2025-07-21','2025-08-15','2025-11-01','2025-11-11','2025-12-25']),
  'lower_window': 0,'upper_window': 0,})

holidays_DE = pd.DataFrame({'holiday': 'holidays_DE',
  'ds': pd.to_datetime(['2022-01-01','2022-04-15','2022-04-18','2022-05-01','2022-05-26','2022-06-06','2022-10-03','2022-12-25','2022-12-26',
                        '2023-01-01','2023-04-07','2023-04-10','2023-05-01','2023-05-18','2023-05-29','2023-10-03','2023-12-25','2023-12-26',
                        '2024-01-01','2024-03-29','2024-04-01','2024-05-01','2024-05-09','2024-05-20','2024-10-03','2024-12-25','2024-12-26',
                        '2025-01-01','2025-04-18','2025-04-21','2025-05-01','2025-05-29','2025-06-09','2025-10-03','2025-12-25','2025-12-26']),
  'lower_window': 0,'upper_window': 0,})

holidays_DK = pd.DataFrame({'holiday': 'holidays_DK',
  'ds': pd.to_datetime(['2022-01-01','2022-04-14','2022-04-15','2022-04-18','2022-05-13','2022-05-26','2022-06-05','2022-06-06','2022-12-24','2022-12-25','2022-12-26',
                        '2023-01-01','2023-04-06','2023-04-07','2023-04-10','2023-05-05','2023-05-18','2023-05-28','2023-05-29','2023-12-24','2023-12-25','2023-12-26',
                        '2024-01-01','2024-03-28','2024-03-29','2024-04-01','2024-05-09','2024-05-19','2024-05-20','2024-12-24','2024-12-25','2024-12-26',
                        '2025-01-01','2025-04-17','2025-04-18','2025-04-21','2025-05-29','2025-06-08','2025-06-09','2025-12-24','2025-12-25','2025-12-26']),
  'lower_window': 0,'upper_window': 0,})

holidays_ES = pd.DataFrame({'holiday': 'holidays_ES',
  'ds': pd.to_datetime(['2022-01-01','2022-01-06','2022-04-15','2022-05-01','2022-08-15','2022-10-12','2022-11-01','2022-12-06','2022-12-08','2022-12-25',
                        '2023-01-01','2023-01-06','2023-04-07','2023-05-01','2023-08-15','2023-10-12','2023-11-01','2023-12-06','2023-12-08','2023-12-25',
                        '2024-01-01','2024-01-06','2024-03-29','2024-05-01','2024-08-15','2024-10-12','2024-11-01','2024-12-06','2024-12-08','2024-12-25',
                        '2025-01-01','2025-01-06','2025-04-18','2025-05-01','2025-08-15','2025-10-12','2025-11-01','2025-12-06','2025-12-08','2025-12-25']),
  'lower_window': 0,'upper_window': 0,})

holidays_FI = pd.DataFrame({'holiday': 'holidays_FI',
  'ds': pd.to_datetime(['2022-01-01','2022-01-06','2022-04-15','2022-04-18','2022-05-01','2022-05-08','2022-05-26','2022-06-24','2022-06-25','2022-11-05','2022-11-13','2022-12-06','2022-12-24','2022-12-25','2022-12-26',
                        '2023-01-01','2023-01-06','2023-04-07','2023-04-10','2023-05-01','2023-05-14','2023-05-18','2023-06-23','2023-06-24','2023-11-04','2023-11-12','2023-12-06','2023-12-24','2023-12-25','2023-12-26',
                        '2024-01-01','2024-01-06','2024-03-29','2024-04-01','2024-05-01','2024-05-09','2024-05-12','2024-06-21','2024-06-22','2024-11-02','2024-11-10','2024-12-06','2024-12-24','2024-12-25','2024-12-26',
                        '2025-01-01','2025-01-06','2025-04-18','2025-04-21','2025-05-01','2025-05-11','2025-05-29','2025-06-20','2025-06-21','2025-11-01','2025-11-09','2025-12-06','2025-12-24','2025-12-25','2025-12-26']),
  'lower_window': 0,'upper_window': 0,})

holidays_FR = pd.DataFrame({'holiday': 'holidays_FR',
  'ds': pd.to_datetime(['2022-01-01','2022-04-18','2022-05-01','2022-05-08','2022-05-26','2022-06-06','2022-07-14','2022-08-15','2022-11-01','2022-11-11','2022-12-25',
                        '2023-01-01','2023-04-10','2023-05-01','2023-05-08','2023-05-18','2023-05-29','2023-07-14','2023-08-15','2023-11-01','2023-11-11','2023-12-25',
                        '2024-01-01','2024-04-01','2024-05-01','2024-05-08','2024-05-09','2024-05-20','2024-07-14','2024-08-15','2024-11-01','2024-11-11','2024-12-25',
                        '2025-01-01','2025-04-21','2025-05-01','2025-05-08','2025-05-29','2025-06-09','2025-07-14','2025-08-15','2025-11-01','2025-11-11','2025-12-25']),
  'lower_window': 0,'upper_window': 0,})

holidays_GB = pd.DataFrame({'holiday': 'holidays_GB',
  'ds': pd.to_datetime(['2022-01-01','2022-01-03','2022-04-15','2022-05-02','2022-06-02','2022-06-03','2022-09-19','2022-12-25','2022-12-26','2022-12-27',
                        '2023-01-01','2023-01-02','2023-04-07','2023-05-01','2023-05-08','2023-05-29','2023-12-25','2023-12-26',
                        '2024-01-01','2024-03-29','2024-05-06','2024-05-27','2024-12-25','2024-12-26',
                        '2025-01-01','2025-04-18','2025-05-05','2025-05-26','2025-12-25','2025-12-26']),
  'lower_window': 0,'upper_window': 0,})

holidays_IE = pd.DataFrame({'holiday': 'holidays_IE',
  'ds': pd.to_datetime(['2022-01-01','2022-03-17','2022-03-18','2022-04-18','2022-05-02','2022-06-06','2022-08-01','2022-10-31','2022-12-25','2022-12-26',
                        '2023-01-01','2023-02-06','2023-03-17','2023-04-10','2023-05-01','2023-06-05','2023-08-07','2023-10-30','2023-12-25','2023-12-26',
                        '2024-01-01','2024-02-05','2024-03-17','2024-04-01','2024-05-06','2024-06-03','2024-08-05','2024-10-28','2024-12-25','2024-12-26',
                        '2025-01-01','2025-02-03','2025-03-17','2025-04-21','2025-05-05','2025-06-02','2025-08-04','2025-10-27','2025-12-25','2025-12-26']),
  'lower_window': 0,'upper_window': 0,})

holidays_NL = pd.DataFrame({'holiday': 'holidays_NL',
  'ds': pd.to_datetime(['2022-01-01','2022-04-17','2022-04-18','2022-04-27','2022-05-26','2022-06-05','2022-06-06','2022-12-25','2022-12-26',
                        '2023-01-01','2023-04-09','2023-04-10','2023-04-27','2023-05-18','2023-05-28','2023-05-29','2023-12-25','2023-12-26',
                        '2024-01-01','2024-03-31','2024-04-01','2024-04-27','2024-05-09','2024-05-19','2024-05-20','2024-12-25','2024-12-26',
                        '2025-01-01','2025-04-20','2025-04-21','2025-04-26','2025-05-05','2025-05-29','2025-06-08','2025-06-09','2025-12-25','2025-12-26']),
  'lower_window': 0,'upper_window': 0,})

holidays_PT = pd.DataFrame({'holiday': 'holidays_PT',
  'ds': pd.to_datetime(['2022-01-01','2022-04-15','2022-04-17','2022-04-25','2022-05-01','2022-06-10','2022-06-16','2022-08-15','2022-10-05','2022-11-01','2022-12-01','2022-12-08','2022-12-25',
                        '2023-01-01','2023-04-07','2023-04-09','2023-04-25','2023-05-01','2023-06-08','2023-06-10','2023-08-15','2023-10-05','2023-11-01','2023-12-01','2023-12-08','2023-12-25',
                        '2024-01-01','2024-03-29','2024-03-31','2024-04-25','2024-05-01','2024-05-30','2024-06-10','2024-08-15','2024-10-05','2024-11-01','2024-12-01','2024-12-08','2024-12-25',
                        '2025-01-01','2025-04-18','2025-04-20','2025-04-25','2025-05-01','2025-06-10','2025-06-19','2025-08-15','2025-10-05','2025-11-01','2025-12-01','2025-12-08','2025-12-25']),
  'lower_window': 0,'upper_window': 0,})

holidays_SE = pd.DataFrame({'holiday': 'holidays_SE',
  'ds': pd.to_datetime(['2022-01-01','2022-01-06','2022-04-15','2022-04-17','2022-04-18','2022-05-01','2022-05-26','2022-06-05','2022-06-06','2022-06-24','2022-06-25','2022-11-05','2022-12-24','2022-12-25','2022-12-26','2022-12-31',
                        '2023-01-01','2023-01-06','2023-04-07','2023-04-09','2023-04-10','2023-05-01','2023-05-18','2023-05-28','2023-06-06','2023-06-23','2023-06-24','2023-11-04','2023-12-24','2023-12-25','2023-12-26','2023-12-31',
                        '2024-01-01','2024-01-06','2024-03-29','2024-03-31','2024-04-01','2024-05-01','2024-05-09','2024-05-19','2024-06-06','2024-06-21','2024-06-22','2024-11-02','2024-12-24','2024-12-25','2024-12-26','2024-12-31',
                        '2025-01-01','2025-01-06','2025-04-18','2025-04-20','2025-04-21','2025-05-01','2025-05-29','2025-06-06','2025-06-08','2025-06-20','2025-06-21','2025-11-01','2025-12-24','2025-12-25','2025-12-26','2025-12-31']),
  'lower_window': 0,'upper_window': 0,})

holidays_OTHER = pd.DataFrame({'holiday': 'holidays_OTHER',
  'ds': pd.to_datetime(['2022-01-01','2023-01-01','2024-01-01','2024-12-25','2025-01-01','2025-12-25']),
  'lower_window': 0,'upper_window': 0,})


# **Spend Forecast - Prophet parameters**

In [ ]:
import logging, sys

changepoint = 0.1 #https://facebook.github.io/prophet/docs/trend_changepoints.html

n_periods = 210 # amount of days to forecast. 210 as the result of forecasting 150 plus 90 we use for test / train

interval_width = 0.85 #the width of the uncertainty intervals (805 of the samples should fit between these boundaries)

growth_spend = 'linear' #Flat, Linear, Logistic. The linear assumes an in(de)creasing trend whereas the flat does not.
#PR for adding more growth trends in Prophet:  https://github.com/facebook/prophet/pull/1466/files

fourier_order_yearly = 10 # determines how quickly the seasonality can change. By default for yearly is 10, monthly is 5 and weekly is 3. It can lead to overfitting.
fourier_order_weekly = 7
fourier_order_monthly = 5
# https://medium.com/analytics-vidhya/how-does-prophet-work-part-2-c47a6ceac511

In [ ]:
df_spend = df1.drop(['spending_customer_count', 'spend_per_spending_customer'], axis=1)
df_spend = df_spend.rename(columns={'performed_date': 'ds', 'spend': 'y'})
df_spend.head()

,ds,country,y
0,2024-04-18,ES,7.985397e+05
1,2022-05-19,GB,2.587917e+06
2,2023-01-09,Other Markets,1.244280e+05
3,2021-12-31,DK,6.346597e+05
4,2023-03-25,GB,2.469190e+06


In [ ]:
from datetime import datetime

split_date = pd.to_datetime(datetime.now().date() - timedelta(days=90))

split_date

Timestamp('2024-06-26 00:00:00')

In [ ]:
# Using this split to add an error multiplier later on to adjust accuracy
train_df_spend = df_spend.loc[df_spend['ds'] < split_date]
test_df_spend = df_spend.loc[df_spend['ds'] >= split_date]

train_df_spend.head()

,ds,country,y
0,2024-04-18,ES,7.985397e+05
1,2022-05-19,GB,2.587917e+06
2,2023-01-09,Other Markets,1.244280e+05
3,2021-12-31,DK,6.346597e+05
4,2023-03-25,GB,2.469190e+06


# **Spend Forecast - Prophet loop**

In [ ]:
p = list()

# Start time
start_time = pd.Timestamp('now')
print('start_time = ',start_time) #print('start_time = ',pd.Timestamp('now'))

for country in train_df_spend['country'].unique():
            logging.disable(sys.maxsize)

            train = train_df_spend.loc[(train_df_spend['country'] == country) ]
            test = test_df_spend.loc[(test_df_spend['country'] == country)]

            holidays = pd.DataFrame ()
            if country == 'AT':
                holidays = holidays_AT
            elif country == 'BE':
                holidays = holidays_BE
            elif country == 'DE':
                holidays = holidays_DE
            elif country == 'DK':
                holidays = holidays_DK
            elif country == 'ES':
                holidays = holidays_ES
            elif country == 'FI':
                holidays = holidays_FI
            elif country == 'FR':
                holidays = holidays_FR
            elif country == 'GB':
                holidays = holidays_GB
            elif country == 'IE':
                holidays = holidays_IE
            elif country == 'NL':
                holidays = holidays_NL
            elif country == 'PT':
                holidays = holidays_PT
            elif country == 'SE':
                holidays = holidays_SE
            else:
                holidays = holidays_OTHER


            #handling error if dataset not ready for training ("ValueError: Dataframe has less than 2 non-NaN rows.")
            if train.shape[0] < 2:
                continue

            m = Prophet(growth = growth_spend, yearly_seasonality=True, weekly_seasonality=True, interval_width=interval_width)

            if country != 'OTHER':

                m = Prophet(yearly_seasonality=True, weekly_seasonality=True, interval_width = interval_width, holidays=holidays)
            else:
                m = Prophet(yearly_seasonality=True, weekly_seasonality=True, interval_width = interval_width)

            m.add_seasonality(name='weekly', period = 7, fourier_order = fourier_order_weekly)
            m.add_seasonality(name='monthly', period = 30, fourier_order = fourier_order_monthly)
            m.add_seasonality(name='yearly', period = 12, fourier_order = fourier_order_yearly)

            m.fit(train)

            # Setting the forecast parameters
            future_df = m.make_future_dataframe(periods = n_periods , include_history=True)
            future_df['country'] = country

            forecast_df = m.predict(future_df)

            forecast_df['country'] = country

            p.append(forecast_df[['ds', 'yhat', 'country']])
    #forecast.tail()

end_time = pd.Timestamp('now')
print('end_time = ',end_time)
print('The time used for the for-loop forecast is ', end_time-start_time)

start_time =  2024-09-24 07:57:53.536882
end_time =  2024-09-24 07:57:59.014265
The time used for the for-loop forecast is  0 days 00:00:05.477383


In [ ]:
p_backup = p #creating a backup dataset

p = pd.concat(p, ignore_index=True)
p['yhat'] = p['yhat'].clip(lower=0) #excluding records where the forecast < 0

print("The starting date of the forecast output is ", p["ds"].min(), " and the last date of the forecast output is ", p["ds"].max() )

The starting date of the forecast output is  2021-02-01 00:00:00  and the last date of the forecast output is  2025-01-21 00:00:00


In [ ]:
p.describe()

,ds,yhat
count,11404,1.140400e+04
mean,2023-02-07 06:27:08.902139648,7.211249e+05
min,2021-02-01 00:00:00,0.000000e+00
25%,2022-02-17 00:00:00,9.427643e+04
50%,2023-02-09 00:00:00,4.037128e+05
75%,2024-01-31 00:00:00,1.016384e+06
max,2025-01-21 00:00:00,4.649281e+06
std,NaN,8.803832e+05


# **Forecast Accuracy**

In [ ]:
# Importing statistical modules

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Creating df with metrics to evaluate model accuracy

metric_df = p.merge(test_df_spend, how="outer", on=['ds', 'country'])

metric_df = metric_df.rename(columns={'yhat': 'spend_forecast', 'y': 'spend_actual'})

metric_df['error_spend'] = round( metric_df['spend_forecast'] / metric_df['spend_actual'], 3) # how much bigger or lower is the forecast compared to the actuals ?

metric_df['day_of_week'] = metric_df['ds'].dt.dayofweek

metric_df.dropna(inplace=True)
metric_df.head()

,ds,spend_forecast,country,spend_actual,error_spend,day_of_week
1231,2024-06-26,674056.246814,ES,748526.74,0.901,2
1232,2024-06-27,688189.164226,ES,804775.50,0.855,3
1233,2024-06-28,685628.927292,ES,769922.49,0.891,4
1234,2024-06-29,635269.528225,ES,670441.13,0.948,5
1235,2024-06-30,452175.729044,ES,297576.90,1.520,6


In [ ]:
print("The starting date of the train output is ", metric_df["ds"].min(), " and the last date of the train output is ", metric_df["ds"].max() )
print("The starting date of the forecast output is ", p["ds"].min(), " and the last date of the forecast output is ", p["ds"].max() )

#metric_df is the df that I use to calculate the forecast error.
#This contains 90 days prior to the current date as declared in the train / test split
#The error multiplier that I create here should be applied to the actual forecast which is p


The starting date of the train output is  2024-06-26 00:00:00  and the last date of the train output is  2024-09-24 00:00:00
The starting date of the forecast output is  2021-02-01 00:00:00  and the last date of the forecast output is  2025-01-21 00:00:00


In [ ]:
print("The overall R2 SCORE accuracy is ", round( r2_score (metric_df.spend_forecast, metric_df.spend_actual ), 2))
print("The overall MSE is ", round( mean_squared_error (metric_df.spend_forecast, metric_df.spend_actual ), 2))
print("The overall MAE is ", round( mean_absolute_error (metric_df.spend_forecast, metric_df.spend_actual ), 2))

# R2 tells about the correlation between two datasets while MSE talks about the differente between them.


The overall R2 SCORE accuracy is  0.89
The overall MSE is  127884819020.48
The overall MAE is  199895.32


In [ ]:
error_prep = metric_df[['country','day_of_week','error_spend']]

error_df = error_prep.groupby(['country','day_of_week']).mean().reset_index()

# **Adding Multiplier**

In [ ]:
p["day_of_week"] = p['ds'].dt.dayofweek

#joining the average error per week day
p_adjusted = p.merge(error_df, how="outer", on=['country', 'day_of_week'])

p_adjusted["yhat"] = p_adjusted["yhat"] / p_adjusted["error_spend"]

p_adjusted = p_adjusted.drop(['day_of_week', 'error_spend'], axis=1)

p_adjusted

,ds,yhat,country
0,2021-02-01,0.000000,ES
1,2021-02-08,0.000000,ES
2,2021-02-15,0.000000,ES
3,2021-02-22,0.000000,ES
4,2021-03-01,0.000000,ES
...,...,...,...
11399,2024-12-22,39758.410938,FR
11400,2024-12-29,40776.687107,FR
11401,2025-01-05,40658.536555,FR
11402,2025-01-12,42856.104128,FR


# **Push data to BQ**

In [ ]:
p_adjusted['month'] = p_adjusted['ds'].dt.strftime('%Y-%m-01')
p_adjusted["run_date"] = pd.Timestamp('now')#.date()

p_adjusted["month"] = pd.to_datetime(p_adjusted["month"])#.strftime('%Y-%m-%d')
p_adjusted["run_date"] = pd.to_datetime(p_adjusted["run_date"])#.strftime('%Y-%m-%d')

p_adjusted["category"] = "ALL" # The model

#Renaming according to the existing materialized table in BQ
#finance_external_sources.spend_forecast_per_country_only_model_results

p_adjusted = p_adjusted.rename(columns={"spend_type": "product"})

desired_order = ['ds', 'yhat', 'country', 'month', 'run_date']

p_adjusted = p_adjusted[desired_order]

In [ ]:
p_adjusted["month"] = pd.to_datetime(p_adjusted["month"])#.strftime('%Y-%m-%d')
p_adjusted.dtypes

ds          datetime64[ns]
yhat               float64
country             object
month       datetime64[ns]
run_date    datetime64[ns]
dtype: object

In [ ]:
p['month'] = p['ds'].dt.strftime('%Y-%m-01')
p["run_date"] = pd.Timestamp('now')#.date()

p["month"] = pd.to_datetime(p["month"])#.strftime('%Y-%m-%d')
p["run_date"] = pd.to_datetime(p["run_date"])#.strftime('%Y-%m-%d')

p["category"] = "ALL" # The model

#Renaming according to the existing materialized table in BQ
#finance_external_sources.spend_forecast_per_country_only_model_results

p = p.rename(columns={"spend_type": "product"})

desired_order = ['ds', 'yhat', 'country', 'month', 'run_date']

p = p[desired_order]

p

,ds,yhat,country,month,run_date
0,2021-02-01,0.000000,ES,2021-02-01,2024-09-24 07:57:59.348174
1,2021-02-02,34478.988747,ES,2021-02-01,2024-09-24 07:57:59.348174
2,2021-02-03,56768.244301,ES,2021-02-01,2024-09-24 07:57:59.348174
3,2021-02-04,65547.633568,ES,2021-02-01,2024-09-24 07:57:59.348174
4,2021-02-05,60346.240428,ES,2021-02-01,2024-09-24 07:57:59.348174
...,...,...,...,...,...
11399,2025-01-17,141259.963329,FR,2025-01-01,2024-09-24 07:57:59.348174
11400,2025-01-18,131340.554838,FR,2025-01-01,2024-09-24 07:57:59.348174
11401,2025-01-19,111736.613766,FR,2025-01-01,2024-09-24 07:57:59.348174
11402,2025-01-20,121243.437774,FR,2025-01-01,2024-09-24 07:57:59.348174


In [ ]:
 #print(forecast)
month_fc_df = p_adjusted
month_fc_df['month'] = month_fc_df['ds'].dt.strftime('%Y-%m-01')
month_fc_df.rename(columns={"yhat": "Spend"}, inplace=True)

month_fc_df[['month', 'Spend']].groupby(['month']).sum().tail()

,Spend
month,
2024-09-01,2.634291e+08
2024-10-01,2.878063e+08
2024-11-01,2.857000e+08
2024-12-01,2.643390e+08
2025-01-01,1.795759e+08


In [ ]:
# Current run
# Pushing the Non Adjusted

pandas_gbq.to_gbq(p, 'finance_external_sources.spend_forecast_per_country_only_model_results', "production", if_exists = "append") #replace")

print(' Completed')

100%|██████████| 1/1 [00:00<00:00, 7724.32it/s]

 Completed


In [ ]:
p

,ds,yhat,country,month,run_date
0,2021-02-01,0.000000,ES,2021-02-01,2024-09-24 07:57:59.348174
1,2021-02-02,34478.988747,ES,2021-02-01,2024-09-24 07:57:59.348174
2,2021-02-03,56768.244301,ES,2021-02-01,2024-09-24 07:57:59.348174
3,2021-02-04,65547.633568,ES,2021-02-01,2024-09-24 07:57:59.348174
4,2021-02-05,60346.240428,ES,2021-02-01,2024-09-24 07:57:59.348174
...,...,...,...,...,...
11399,2025-01-17,141259.963329,FR,2025-01-01,2024-09-24 07:57:59.348174
11400,2025-01-18,131340.554838,FR,2025-01-01,2024-09-24 07:57:59.348174
11401,2025-01-19,111736.613766,FR,2025-01-01,2024-09-24 07:57:59.348174
11402,2025-01-20,121243.437774,FR,2025-01-01,2024-09-24 07:57:59.348174


In [ ]:
#p.to_csv('guille_spend_fct_0506.csv', index=False)

In [ ]:
#p_adjusted.to_csv('guille_spend_fct_adjusted_2805.csv', index=False)